$$\newcommand{\ket}[1]{\left|{#1}\right\rangle}$$
$$\newcommand{\bra}[1]{\left\langle{#1}\right|}$$
$$\newcommand{\braket}[2]{\left\langle{#1}\middle|{#2}\right\rangle}$$

# Teleporting an Unknown Quantum State via Dual Einstein-Podolsky-Rosen Channels

*Physical Review Letters, 70:13, 1895. 29 March 1993*

https://researcher.watson.ibm.com/researcher/files/us-bennetc/BBCJPW.pdf.

*Nielsen and Chuang, section 1.3.7, page 26*

Alice and Bob met long ago but now live far apart. While together they generated an EPR pair of entangled particles (particles two and three), each taking one qubit of the pair when they separated. Many years later, Bob is in hiding, and Alice’s mission is to precisely deliver a qubit (the state of particle one) $\ket{\Psi}$ to Bob. She does not know the state of the qubit, and moreover can only send classical information to Bob.

This looks impossible; she doesn't know the state of particle one and because of quantum mechanics, she cannot determine it (measurement can only give her a single classical bit - one or zero). Besides that, even if she knew the values of amplitudes of the qubit, she would need an infinite number of classical bits to precisely communicate them.

Here is how they do it:
<ol>
    <li> Alice entangles particles two and three. Partice one, the qubit, is in an unknown state</li>
    <li> Bob takes particle three to a distant galaxy</li>
    <li> Alice measures the state of the qubit (particle one) and her particle (particle two), getting two classical bits of information</li>
    <li> Alice send the classical bits to Bob</li>
    <li> Bob uses the classical bits to select one of four physical operations</li>
    <li> Bob performs the operation on his particle (particle three). These operations are simulated here using quantum computing gates</li>
</ol>

The result is that his particle is now in the exact state that particle one was in before Alice measured it.

There are thre points to note:
<ul>
    <li> After Alice and Bob separated, Alice does not need to send anything to Bob except for the two classical bits</li>
    <li> Bob cannot reconstruct the teleported state instantaneously because he has to wait for the classical bits to arrive</li>
    <li> The speed of transmission is limited by the speed of light. There is no "telepathy."
</ul>

<img src="bobs-operations.jpg" style="width:8in;"/>

Because Alice is using a measurement basis consisting of the Bell operator state, there is a one-to-one corresondence between those states and her measurement results.

Bob chooses his physical operation according to the results of Alice's measurement according to the following table:

| Alice's measurement | Bob's physical operation/quantum gate |
| ------------- | ---------- |
| $\Psi^+\rightarrow \ket{00}$ | none |
| $\Psi^-\rightarrow \ket{10}$ | Z |
| $\Phi^+\rightarrow \ket{01}$ | X |
| $\Phi^-\rightarrow \ket{11}$ | X |


## Notes on Measurement

<ul>
    <li> Measuring a qubit yields a classical bit</li>
    <li> Before a measurement is done, a measurement basis has to be chosen</li>
    <li> Choosing a measurement basis is like choosing orientation of a polarizing filter for light, or orienting a magnetic field for electrons</li>
    <li> In a quantum computer, measurement basis is chosen by applying a quantum gate called a "unitary gate."</li>
    <li> To apply a unitary gate, you create a unitary matrix and pass it to the IBM software (the API). In a real quantum computer system, the associated electronics would prepare an appropriate microwave pulse and send it in.
    </li> After applying the gate, a standard measurement is performed, getting one or zero, but the outcome is determined by the probability of the measured state being in one of the basis state.
    <li> Measurement basis states do not have to be limited to single-qubit states. In this experiment, we are working with entanglements of two qubits (Alice's particles)
</ul>
    

In [1]:
import qiskit
from qiskit import QuantumCircuit, transpile, execute
from qiskit.extensions import UnitaryGate
from qiskit.providers.aer import QasmSimulator
from qiskit.tools.visualization import plot_bloch_multivector, plot_histogram
from qiskit import IBMQ
from qiskit.tools.monitor import job_monitor

import numpy as np
import matplotlib.pyplot as plt
import warnings
import math
warnings.filterwarnings('ignore')

In [2]:
#Utilities for displaying states from the state vector simulator
# Two particles
def show_statevector(statevector):
    print(f'statevector\nuu, du, ud, dd:\n{np.around(statevector[0],3)} {np.around(statevector[1],3)} \
     {np.around(statevector[2], 3)} {np.around(statevector[3],3)}')
def show_statevector3(statevector, title=''):
    #Three particles
    print(f'title\
    uuu:{np.around(statevector[0],3)} duu:{np.around(statevector[1],3)}\
     udu:{np.around(statevector[2], 3)} ddu:{np.around(statevector[3],3)}\
     uud:{np.around(statevector[4], 3)} dud:{np.around(statevector[5],3)}\
     udd:{np.around(statevector[6], 3)} ddd{np.around(statevector[7], 3)}')
statevector_simulator = qiskit.Aer.get_backend('statevector_simulator')


### Interpretation of displayed state vectors

Single-qubit states

$(1,0)\rightarrow\ket{u}\rightarrow\ket{0}$

$(0,1)\rightarrow\ket{d}\rightarrow\ket{1}$

Two-qubit states - note that this convention is different from that used by Neilsen and Chuang

$(1,0,0,0)\rightarrow \ket{uu}\rightarrow\ket{00}$

$(0,1,0,0)\rightarrow \ket{du}\rightarrow\ket{10}$

$(0,0,1,0)\rightarrow \ket{ud}\rightarrow\ket{01}$

$(0,0,0,0)\rightarrow \ket{dd}\rightarrow\ket{11}$


## Alice's Measurement basis

### Bell operator states (EPR states) - Nielsen and Chuang, section 1.3.6 page 25

$\Psi^+=\sqrt{\frac{1}{2}}(\ket{ud}+\ket{du}=(1,0,0,0))$

$\Psi^-=\sqrt{\frac{1}{2}}(\ket{ud}-\ket{du}=(0,1,0,0))$

$\ket{\Phi^+}=\sqrt{\frac{1}{2}}(\ket{uu}+\ket{dd}=(0,0,1,0)$

$\ket{\Phi^-}=\sqrt{\frac{1}{2}}(\ket{uu}-\ket{dd}=(0,0,0,1)$

$U=\begin{bmatrix}\ket{\Psi^+}\\\\\ket{\Psi^-}\\\\\ket{\Phi^+}\\\\\ket{\Phi^-}\end{bmatrix}$


In [3]:
def measurement_gate(show=False):
    u=np.array([[0,1,1,0],\
                 [0,-1,1,0],\
                 [1,0,0,1],\
                  [1,0,0,-1]])/np.sqrt(2)

    if show:
        print('unitary matrix for projection')
        print(u[0,0],u[0,1], u[0,2], u[0,3])
        print(u[1,0],u[1,1], u[1,2], u[1,3])
        print(u[2,0], u[2,1], u[2,2], u[2,3])
        print(u[3,0], u[3,1], u[3,2], u[3,3])

    return u    

## Verification that Alice's Measurements are Working Correctly

### $\ket{\Psi^-}$

In [4]:
circuit=qiskit.QuantumCircuit(2,2)
simulator = qiskit.Aer.get_backend('statevector_simulator')

# Prepare psi-
circuit.h(0)
circuit.cx(0,1)
circuit.x(0)
circuit.z(0)

circuit.unitary(measurement_gate(),[0,1])
circuit.measure(1,1)
result = execute(circuit, backend=simulator, shots=1).result() 
statevector = result.get_statevector()
show_statevector(statevector)

statevector
uu, du, ud, dd:
(-0+0j) (1-0j)      0j 0j


### $\ket{\Phi^+}$

In [5]:
circuit=qiskit.QuantumCircuit(2,2)
simulator = qiskit.Aer.get_backend('statevector_simulator')

# Prepare phi+: uu+dd
circuit.h(0)
circuit.cx(0,1)

circuit.unitary(measurement_gate(),[0,1])
result = execute(circuit, backend=simulator, shots=1).result() 
statevector = result.get_statevector()
show_statevector(statevector)

statevector
uu, du, ud, dd:
0j 0j      (1+0j) 0j


### Alice entangles particles two and three before Bob leaves, Making $\ket{\Psi_{23}^(-)}$

The qubit index for particle one is zero, the index for particle two is one and the index for particle three is two.

In [6]:
def entangle():
    circuit=qiskit.QuantumCircuit(3,3)
    circuit.h(1)
    circuit.cx(1,2)
    circuit.x(1)
    circuit.z(1)
    return circuit


In [7]:
# Verify that the function entangle produces the correct state
verify = True
if verify:
    circuit=entangle()
    circuit.draw()
    simulator = qiskit.Aer.get_backend('statevector_simulator')
    result = execute(circuit, backend=simulator, shots=1).result() 
    statevector = result.get_statevector()
    show_statevector3(statevector)

title    uuu:0j duu:0j     udu:(-0.707+0j) ddu:0j     uud:(0.707+0j) dud:0j     udd:0j ddd0j


## States before any measurements

Particle one: $\phi_1 = (a,b)$

Combined state of three particles:

$\ket{\Psi_{123}}=\ket{\phi}\ket{\Psi_{23}^{(-)}}$ Particles two and three are entangled but particle one is not

Substituting for $\ket{\phi}$ and $\Psi_{23}^{(-)}$:

(Equation 5 from paper) $\ket{\Psi_{123}}=\frac{1}{2}[\ket{\psi_{12}^-}(-a\ket{u_3}-b\ket{d_3})+$
$\ket{\Psi_{12}^+}(-a\ket{u_3}+b\ket{d_3}+$
$\ket{\Phi_{12}^-}(a\ket{d_3}+b\ket{u_3})+$
$\ket{\Phi_{12}+}(a\ket{d_3}-b\ket{u_3})]$

## First Experiment: Particle One is in a Pure Spin-up State

In [8]:
circuit = entangle()

## Alice's Measures Particles One and Two

### Measurement basis: Bell operator states (EPR states) - Nielsen and Chuang, section 1.3.6 page 25

$\Psi^+=\sqrt{\frac{1}{2}}(\ket{ud}+\ket{du}=(1,0,0,0))$

$\Psi^-=\sqrt{\frac{1}{2}}(\ket{ud}-\ket{du}=(0,1,0,0))$

$\ket{\Phi^+}=\sqrt{\frac{1}{2}}(\ket{uu}+\ket{dd}=(0,0,1,0)$

$\ket{\Phi^-}=\sqrt{\frac{1}{2}}(\ket{uu}-\ket{dd}=(0,0,0,1)$

### Quantum computing gate for Alice's measurement

$U=\begin{bmatrix}\ket{\Psi^+}\\\\\ket{\Psi^-}\\\\\ket{\Phi^+}\\\\\ket{\Phi^-}\end{bmatrix} \text{computation basis}=\sqrt{\frac{1}{2}}\begin{bmatrix}
0&1&1&0\\
0&-1&1&0\\
1&0&0&1\\
1&0&0&-1\end{bmatrix}$

 Equation (5) with $a=1$ and $b=0$:
 
 $\ket{\Psi_{123}}=\frac{1}{2}[\ket{\Psi_{12}^{(-)}}(-\ket{u})+$
 $\ket{\Psi_{12}^{(+)}}(-\ket{u})+$
 $\ket{\Phi_{12}^{(-)}}\ket{d}$
 $+\ket{\Phi_{12}^{(+)}}\ket{d}]\rightarrow \frac{1}{2}[-\ket{duu}-\ket{uuu}+\ket{ddd}+\ket{udd}]$

In [9]:
circuit.unitary(measurement_gate(),[0,1])
simulator = qiskit.Aer.get_backend('statevector_simulator')

result = execute(circuit, backend=simulator, shots=1).result() 
statevector = result.get_statevector()
show_statevector3(statevector)

title    uuu:(-0.5+0j) duu:(-0.5+0j)     udu:0j ddu:0j     uud:0j dud:0j     udd:(0.5+0j) ddd(0.5+0j)


### Reconstructing the particle state

| Partice one state | Particle's One and Two after measurement|Alice's measurement | Particle three state before operation | Particle three final state|
| ------------- | ---------- | ----- | ---- |----- |
| $\ket{u}$ | $\ket{\Psi^+}$ | 00 | $-\ket{u}$ |$\ket{u}$|
| $\ket{d}$ | $\ket{\Psi^-}$ | 10 | $-\ket{d}$ |$\ket{d}$|
| $\ket{u}$ | $\ket{\Phi^+}$ | 01 | $\ket{d}$ |$\ket{u}$|
| $\ket{d}$ | $\ket{\Phi^-}$ | 11| $\ket{u}$ |$\ket{d}$|

#### Now we simulate the experiment on the quantum computer, first preparing particle one in the "up" state, then the "down" state and finally in a superposition. In all three cases, Alice's four measurement outcomes are equally probable because she is not extracting any information about the state of particle one. Depending on the result of Alice's measurement, the state of Bob's particle before he does his physical operation, is either the same as or opposite from paticle one. That is why he needs to do a physical operation

In [10]:
def spin(k):
    if k=='0':
        return 'u'
    return 'd'
gates = ['none','x','h']
print("Particle one - Alice's Measurement - Particle three before operation - Count")
for gate in gates:
    # Alice entangles particles two and three
    circuit = entangle()
    if gate == 'x':
        circuit.x(0)
    elif gate == 'h':
        circuit.h(0)

    # Alice measures particles one and two in the Bell operator basis and Bob measures particle three
#     in the standard base (spin up/down)
    circuit.unitary(measurement_gate(),[0,1])
    circuit.measure(0,0)
    circuit.measure(1,1)
    circuit.measure(2,2)
    # There are four equally probable outcomes
    result = execute(circuit, backend=QasmSimulator(), shots=8192*4).result() 
    counts=result.get_counts()
#     print('\n')
#     print(gate, counts)
    if gate == 'none':
        state2 = 'u'
    elif gate == 'x':
        state2 = 'd'
    elif gate == 'h':
        state2 = 'mixed'
    else:
        state2 = '?'
    for k in counts.keys():
        three = spin(k[0])
        two = spin(k[1])
        one = spin(k[2])
        if one=='u' and two=='u':
            state = '00'
        elif one=='u' and two=='d':
            state = '01'
        elif one=='d' and two=='u':
            state = '10'
        else:
            state = '11'
#         state = k + '-' + state
        state+='\t\t\t'+three
        print(f'{state2}\t\t{state}\t\t\t\t {counts[k]}')

    print('\n')

Particle one - Alice's Measurement - Particle three before operation - Count
u		10			u				 8157
u		11			d				 8171
u		00			u				 8305
u		01			d				 8135


d		01			u				 8220
d		11			u				 8186
d		10			d				 8292
d		00			d				 8070


mixed		01			u				 4109
mixed		11			d				 4084
mixed		11			u				 4103
mixed		00			u				 4113
mixed		00			d				 4117
mixed		01			d				 4080
mixed		10			u				 4032
mixed		10			d				 4130




#### After Bob has applied the physical operation to reconstruct the teleported state, he can perform measurements in any basis at all - not just spin up/down. The results he gets will have the same distributions of ones and zeros as Alice would have gotten had she measured particle one rather than entangling it.

